In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
solver = po.SolverFactory("gurobi")
model = pe.ConcreteModel()

### Sets Lookup Cell

In [3]:
factories = {
    "Ijmuiden",
    "Segal",
    "South Wales",
}

customers = {
    "Bochum",
    "Boenen",
    "Dortmund",
    "Gelsenkirchen",
    "Hagen",
    "Iserlohn",
    "Neuss",
    "Schwerte",
}

rebar_types = {"A", "B", "C"}
long_bar_types = {"Type 1", "Type 2"}


### Parameter Lookup cell

In [4]:
capacity = {
    "Ijmuiden": 12,
    "Segal": 10,
    "South Wales": 28,
}

factory_fixed_cost = {
    "Ijmuiden": 400,
    "Segal": 500,
    "South Wales": 250,
}

factory_variable_cost = {
    "Ijmuiden": 5,
    "Segal": 3,
    "South Wales": 2,
}

transport_fixed_cost = {
    "Ijmuiden": 130,
    "Segal": 150,
    "South Wales": 100,
}

uncut_bar_weight = {  # in tonnes
    "Type 1": 0.1803,
    "Type 2": 0.2404,
}

rebar_weight = {  # in tonnes
    "A": 0.0481,
    "B": 0.0721,
    "C": 0.0841,
}

patterns = { # for reference
    ("Type 1", 1): (3, 0, 0),
    ("Type 1", 2): (2, 1, 0),
    ("Type 1", 3): (0, 2, 0),
    ("Type 1", 4): (0, 1, 1),
    ("Type 1", 5): (0, 0, 2),
    ("Type 1", 6): (2, 0, 1),
    ("Type 2", 1): (5, 0, 0),
    ("Type 2", 2): (3, 1, 0),
    ("Type 2", 3): (3, 0, 1),
    ("Type 2", 4): (2, 2, 0),
    ("Type 2", 5): (1, 1, 1),
    ("Type 2", 6): (1, 0, 2),
    ("Type 2", 7): (1, 2, 0),
    ("Type 2", 8): (0, 1, 2),
    ("Type 2", 9): (0, 2, 1),
}

rebars_from_pattern = {
    ("Type 1", 1, "A"): 3,
    ("Type 1", 1, "B"): 0,
    ("Type 1", 1, "C"): 0,
    ("Type 1", 2, "A"): 2,
    ("Type 1", 2, "B"): 1,
    ("Type 1", 2, "C"): 0,
    ("Type 1", 3, "A"): 0,
    ("Type 1", 3, "B"): 2,
    ("Type 1", 3, "C"): 0,
    ("Type 1", 4, "A"): 0,
    ("Type 1", 4, "B"): 1,
    ("Type 1", 4, "C"): 1,
    ("Type 1", 5, "A"): 0,
    ("Type 1", 5, "B"): 0,
    ("Type 1", 5, "C"): 2,
    ("Type 1", 6, "A"): 2,
    ("Type 1", 6, "B"): 0,
    ("Type 1", 6, "C"): 1,
    ("Type 2", 1, "A"): 5,
    ("Type 2", 1, "B"): 0,
    ("Type 2", 1, "C"): 0,
    ("Type 2", 2, "A"): 3,
    ("Type 2", 2, "B"): 1,
    ("Type 2", 2, "C"): 0,
    ("Type 2", 3, "A"): 3,
    ("Type 2", 3, "B"): 0,
    ("Type 2", 3, "C"): 1,
    ("Type 2", 4, "A"): 2,
    ("Type 2", 4, "B"): 2,
    ("Type 2", 4, "C"): 0,
    ("Type 2", 5, "A"): 1,
    ("Type 2", 5, "B"): 1,
    ("Type 2", 5, "C"): 1,
    ("Type 2", 6, "A"): 1,
    ("Type 2", 6, "B"): 0,
    ("Type 2", 6, "C"): 2,
    ("Type 2", 7, "A"): 1,
    ("Type 2", 7, "B"): 2,
    ("Type 2", 7, "C"): 0,
    ("Type 2", 8, "A"): 0,
    ("Type 2", 8, "B"): 1,
    ("Type 2", 8, "C"): 2,
    ("Type 2", 9, "A"): 0,
    ("Type 2", 9, "B"): 2,
    ("Type 2", 9, "C"): 1,
}

demand = {
    (1, "Bochum", "A"): 2,
    (1, "Boenen", "A"): 4,
    (1, "Dortmund", "A"): 2,
    (1, "Gelsenkirchen", "A"): 5,
    (1, "Hagen", "A"): 19,
    (1, "Iserlohn", "A"): 13,
    (1, "Neuss", "A"): 20,
    (1, "Schwerte", "A"): 4,

    (2, "Bochum", "A"):6,
    (2, "Boenen", "A"):8,
    (2, "Dortmund", "A"):7,
    (2, "Gelsenkirchen", "A"):5,
    (2, "Hagen", "A"):23,
    (2, "Iserlohn", "A"):19,
    (2, "Neuss", "A"):16 ,
    (2, "Schwerte", "A"):5,

    (3, "Bochum", "A"):5,
    (3, "Boenen", "A"):5,
    (3, "Dortmund", "A"):6,
    (3, "Gelsenkirchen", "A"):5,
    (3, "Hagen", "A"):25,
    (3, "Iserlohn", "A"):17,
    (3, "Neuss", "A"):14,
    (3, "Schwerte", "A"):3,

    (4, "Bochum", "A"):3,
    (4, "Boenen", "A"):10,
    (4, "Dortmund", "A"):5,
    (4, "Gelsenkirchen", "A"):5,
    (4, "Hagen", "A"):16,
    (4, "Iserlohn", "A"):14,
    (4, "Neuss", "A"):26,
    (4, "Schwerte", "A"):4,


    (1, "Bochum", "B"): 4,
    (1, "Boenen", "B"): 5,
    (1, "Dortmund", "B"): 4,
    (1, "Gelsenkirchen", "B"): 9,
    (1, "Hagen", "B"): 15,
    (1, "Iserlohn", "B"): 22,
    (1, "Neuss", "B"): 12,
    (1, "Schwerte", "B"): 2,

    (2, "Bochum", "B"):5,
    (2, "Boenen", "B"):8,
    (2, "Dortmund", "B"):5,
    (2, "Gelsenkirchen", "B"):10,
    (2, "Hagen", "B"):33,
    (2, "Iserlohn", "B"):26,
    (2, "Neuss", "B"):23,
    (2, "Schwerte", "B"):8 ,

    (3, "Bochum", "B"):7,
    (3, "Boenen", "B"):12,
    (3, "Dortmund", "B"):8,
    (3, "Gelsenkirchen", "B"):6,
    (3, "Hagen", "B"):31,
    (3, "Iserlohn", "B"):20,
    (3, "Neuss", "B"):30,
    (3, "Schwerte", "B"):2,

    (4, "Bochum", "B"):8,
    (4, "Boenen", "B"):13,
    (4, "Dortmund", "B"):10,
    (4, "Gelsenkirchen", "B"):6,
    (4, "Hagen", "B"):33,
    (4, "Iserlohn", "B"):27,
    (4, "Neuss", "B"):30,
    (4, "Schwerte", "B"):6,

    (1, "Bochum", "C"): 6,
    (1, "Boenen", "C"): 6,
    (1, "Dortmund", "C"): 7,
    (1, "Gelsenkirchen", "C"): 10,
    (1, "Hagen", "C"): 12,
    (1, "Iserlohn", "C"): 14,
    (1, "Neuss", "C"): 22,
    (1, "Schwerte", "C"): 5,

    (2, "Bochum", "C"): 7,
    (2, "Boenen", "C"): 10,
    (2, "Dortmund", "C"): 6,
    (2, "Gelsenkirchen", "C"): 9,
    (2, "Hagen", "C"): 35,
    (2, "Iserlohn", "C"): 25,
    (2, "Neuss", "C"): 32,
    (2, "Schwerte", "C"): 6,

    (3, "Bochum", "C"): 7,
    (3, "Boenen", "C"): 15,
    (3, "Dortmund", "C"): 4,
    (3, "Gelsenkirchen", "C"): 9,
    (3, "Hagen", "C"): 33,
    (3, "Iserlohn", "C"): 23,
    (3, "Neuss", "C"): 31,
    (3, "Schwerte", "C"): 7,

    (4, "Bochum", "C"): 7,
    (4, "Boenen", "C"): 12,
    (4, "Dortmund", "C"): 12,
    (4, "Gelsenkirchen", "C"): 10,
    (4, "Hagen", "C"): 38,
    (4, "Iserlohn", "C"): 24,
    (4, "Neuss", "C"): 31,
    (4, "Schwerte", "C"): 2,
}

transport_distances = {
    ("Ijmuiden", "Bochum"): 250,
    ("Segal", "Bochum"): 203,
    ("South Wales", "Bochum"): 866,
    ("Ijmuiden", "Boenen"): 282,
    ("Segal", "Boenen"): 242,
    ("South Wales", "Boenen"): 914,
    ("Ijmuiden", "Dortmund"): 266,
    ("Segal", "Dortmund"): 222,
    ("South Wales", "Dortmund"): 885,
    ("Ijmuiden", "Gelsenkirchen"): 234,
    ("Segal", "Gelsenkirchen"): 198,
    ("South Wales", "Gelsenkirchen"): 859,
    ("Ijmuiden", "Hagen"): 289,
    ("Segal", "Hagen"): 206,
    ("South Wales", "Hagen"): 903,
    ("Ijmuiden", "Iserlohn"): 299,
    ("Segal", "Iserlohn"): 226,
    ("South Wales", "Iserlohn"): 913,
    ("Ijmuiden", "Neuss"): 259,
    ("Segal", "Neuss"): 140,
    ("South Wales", "Neuss"): 843,
    ("Ijmuiden", "Schwerte"): 279,
    ("Segal", "Schwerte"): 216,
    ("South Wales", "Schwerte"): 901,
}

### Pyomo Model | Sets

In [5]:
model.Factories = pe.Set(initialize=factories, ordered=False)
model.Bars = pe.Set(initialize=long_bar_types, ordered=False)
model.Rebars = pe.Set(initialize=rebar_types, ordered=False)
model.Customers = pe.Set(initialize=customers, ordered=False)

model.Periods = pe.RangeSet(1, 4)
model.PatternsType1 = pe.RangeSet(1, 6)
model.PatternsType2 = pe.RangeSet(1, 9)

### Pyomo Model | Variables

In [6]:
# First, we would like to know which factory is used for producing bars
model.factory_active = pe.Var(
    model.Periods,
    model.Factories,
    domain=pe.Binary,
)

# Next, we track what type of long bar is produced by each factory
model.production_bar = pe.Var(
    model.Periods,
    model.Factories,
    model.Bars,
    domain=pe.NonNegativeIntegers,
)

# Afterwards, we want to know how the factories cut their bars
model.rebars_cut_type1 = pe.Var(
    model.Periods,
    model.Factories,
    model.PatternsType1,
    domain=pe.NonNegativeIntegers,
)

model.rebars_cut_type2 = pe.Var(
    model.Periods,
    model.Factories,
    model.PatternsType2,
    domain=pe.NonNegativeIntegers,
)

# Thereafter, we want to know which customers are served from which factory
model.ships = pe.Var(
    model.Periods,
    model.Factories,
    model.Customers,
    domain=pe.Binary,
)

# Finally, how many rebars is carried for each shipment
model.carries = pe.Var(
    model.Periods,
    model.Factories,
    model.Customers,
    model.Rebars,
    domain=pe.NonNegativeIntegers,
)

### Pyomo Model | Objective

In [7]:
# Cost for having a factory open
cost_factory_open = sum(
    model.factory_active[period, factory] * factory_fixed_cost[factory] for factory in model.Factories for period in model.Periods
)

# variable cost for producing bars
cost_producing_bars = sum(
    model.production_bar[period, factory, bar] * factory_variable_cost[factory] for bar in model.Bars for factory in model.Factories for period in model.Periods
)

# fixed cost for shipping
cost_shipment = sum(
    model.ships[period, factory, customer] * transport_fixed_cost[factory] for customer in model.Customers for factory in model.Factories for period in model.Periods
)

# Variable cost for shipping

cost_per_km_tonnes = sum(sum(model.carries[period, factory, customer, rebar] * rebar_weight[rebar] for rebar in model.Rebars) * transport_distances[factory, customer] * 0.5
                         for customer in model.Customers for factory in model.Factories for period in model.Periods
)

### Total objective
objective = cost_factory_open + cost_producing_bars + cost_shipment + cost_per_km_tonnes
model.obj = pe.Objective(sense=pe.minimize, expr=objective)

### Pyomo Model | Constraints

In [8]:
model.maxProd = pe.ConstraintList()

# You cannot produce more than capacity
for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.production_bar[period, factory, bar] * uncut_bar_weight[bar] for bar in model.Bars) <= capacity[factory]
        model.maxProd.add(expr=expr)
        # print(expr) # Uncomment to see constarint

In [9]:
model.factoryOpen = pe.ConstraintList()

# You can only produce bars if the factory is active
for period in model.Periods:
    for factory in model.Factories:
        expr = sum(model.production_bar[period, factory, bar] for bar in model.Bars) <= model.factory_active[period, factory] * 10000
        
        model.factoryOpen.add(expr=expr)
        # print(expr) # Uncomment to see constarint

In [10]:
model.cuttingBars = pe.ConstraintList()

# the number of patterns that we cut should be equal to the number of bars that we produced
for period in model.Periods:
        for factory in model.Factories:
                
                # Type 1 bar
                expr = (sum(model.rebars_cut_type1[period, factory, pattern] for pattern in model.PatternsType1) ) <= model.production_bar[period, factory, 'Type 1']
                model.cuttingBars.add(expr=expr)
                # print(expr) # Uncomment to see constarint
                
                # Type 2 bar
                expr = (sum(model.rebars_cut_type2[period, factory, pattern] for pattern in model.PatternsType2) ) <= model.production_bar[period, factory, 'Type 2']
                model.cuttingBars.add(expr=expr)
                # print(expr) # Uncomment to see constarint

In [11]:
model.onlyOneFactory = pe.ConstraintList()

# Each customer can only be served by one factory
for period in model.Periods:
    for customer in model.Customers:
        expr = sum(model.ships[period, factory, customer] for factory in model.Factories) == 1
        model.onlyOneFactory.add(expr=expr)
        # print(expr)  # Uncomment to see constarint

In [12]:
model.carryCapacity = pe.ConstraintList()

# A shipment cannot carry more than is produced
for period in model.Periods:
    for factory in model.Factories:
        for rebar in model.Rebars:
            
            available_rebars_from_type1 = sum(model.rebars_cut_type1[period, factory, pattern] * rebars_from_pattern['Type 1', pattern, rebar] for pattern in model.PatternsType1)
            available_rebars_from_type2 = sum(model.rebars_cut_type2[period, factory, pattern] * rebars_from_pattern['Type 2', pattern, rebar] for pattern in model.PatternsType2)
            
            expr = sum(model.carries[period, factory, customer, rebar] for customer in model.Customers) <= available_rebars_from_type1 + available_rebars_from_type2
            
            model.carryCapacity.add(expr=expr)
            # print(expr)  # Uncomment to see constarint

In [13]:
model.meetDemand = pe.ConstraintList()

# Each demand should be met
for period in model.Periods:
    for factory in model.Factories:
        for customer in model.Customers:
            for rebar in model.Rebars:
                expr = model.carries[period, factory, customer, rebar] == model.ships[period, factory, customer] * demand[period, customer, rebar]
            
                model.meetDemand.add(expr=expr)
                # print(expr)  # Uncomment to see constarint

### Pyomo Model | Solver

In [14]:
result = solver.solve(model, tee = True, options = {'TimeLimit':3600})

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-04
Read LP format model from file /var/folders/xk/nwlnsjsx3c321c24ljrms4mc0000gn/T/tmpamta1_4t.pyomo.lp
Reading time = 0.00 seconds
x1: 404 rows, 600 columns, 1548 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 404 rows, 600 columns and 1548 nonzeros
Model fingerprint: 0xda695225
Variable types: 0 continuous, 600 integer (108 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 375 rows and 525 columns
Presolve time: 0.12s
Presolved: 29 rows, 75 columns, 234 nonzeros
Variable types: 0 continuous, 75 integer (27 binary)

Root relaxation: objective 2.196866e+04, 42 it

### Pyomo Model | Results

In [15]:
print(f"objective value is: {pe.value(model.obj)}")

# Which factories where open?
print('\nOpen factories:')
for period in model.Periods:
    for factory in model.Factories:
        if pe.value(model.factory_active[period, factory]) > 0:
            print(f"{period, factory} : {pe.value(model.factory_active[period, factory])}")
    
# how many bars did we produce?
print('\nHow many bars produced:')
for period in model.Periods:
    for factory in model.Factories:
        for bar in model.Bars:
            if pe.value(model.production_bar[period, factory, bar]) > 0:
                print(f"{period, factory, bar} : {pe.value(model.production_bar[period, factory, bar])}")
 
 # how many patterns did we cut?
print('\npatterns from steel bar type 1:')
for period in model.Periods:
    for factory in model.Factories:
        for pattern in model.PatternsType1:
            if pe.value(model.rebars_cut_type1[period, factory, pattern]) > 0:
                print(f"{period, factory, pattern} : {pe.value(model.rebars_cut_type1[period, factory, pattern])}")       

 # how many patterns did we cut?
print('\npatterns from steel bar type 2:')
for period in model.Periods:
    for factory in model.Factories:
        for pattern in model.PatternsType2:
            if pe.value(model.rebars_cut_type2[period, factory, pattern]) > 0:
                print(f"{period, factory, pattern} : {pe.value(model.rebars_cut_type2[period, factory, pattern])}")     
        
        
 # how many patterns did we cut?
print('\nWhich customer was served by which factory?')
for period in model.Periods:
    for customer in model.Customers:
        for factory in model.Factories:
            if pe.value(model.ships[period, factory, customer]) > 0:
                print(f"{period, customer, factory} : {pe.value(model.ships[period, factory, customer])}")     


 # how many patterns did we cut?
print('\nhow much did each shipment carry?')
for period in model.Periods:
    for factory in model.Factories:
        for customer in model.Customers:
            for rebar in model.Rebars:
                if pe.value(model.carries[period, factory, customer, rebar]) > 0:
                    print(f"{period, factory, customer, rebar} : {pe.value(model.carries[period, factory, customer, rebar])}")     

objective value is: 22605.05897008216

Open factories:
(1, 'Segal') : 1.0
(1, 'Ijmuiden') : 1.0
(2, 'Segal') : 1.0
(2, 'South Wales') : 1.0
(2, 'Ijmuiden') : 1.0
(3, 'Segal') : 1.0
(3, 'South Wales') : 1.0
(3, 'Ijmuiden') : 1.0
(4, 'Segal') : 1.0
(4, 'South Wales') : 1.0
(4, 'Ijmuiden') : 1.0

How many bars produced:
(1, 'Segal', 'Type 2') : 40.0
(1, 'Ijmuiden', 'Type 2') : 25.0
(2, 'Segal', 'Type 2') : 40.0
(2, 'Segal', 'Type 1') : 2.0
(2, 'South Wales', 'Type 2') : 9.0
(2, 'South Wales', 'Type 1') : 2.0
(2, 'Ijmuiden', 'Type 2') : 48.0
(3, 'Segal', 'Type 2') : 40.0
(3, 'Segal', 'Type 1') : 2.0
(3, 'South Wales', 'Type 2') : 6.0
(3, 'Ijmuiden', 'Type 2') : 49.0
(3, 'Ijmuiden', 'Type 1') : 1.0
(4, 'Segal', 'Type 2') : 40.0
(4, 'Segal', 'Type 1') : 2.0
(4, 'South Wales', 'Type 2') : 14.0
(4, 'Ijmuiden', 'Type 2') : 49.0

patterns from steel bar type 1:
(2, 'Segal', 2) : 1.0
(2, 'Segal', 6) : 1.0
(2, 'South Wales', 6) : 2.0
(3, 'Segal', 1) : 2.0
(3, 'Ijmuiden', 2) : 1.0
(4, 'Segal', 6) :

### Pyomo Model | Diagnostics

In [16]:
print(f"\nstatus: {result.solver.status}, condition: {result.solver.termination_condition}")


status: ok, condition: optimal
